In [1]:
import os
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import Tk
from PIL import Image

def open_file_dialog():
    # Open dialog to select a file
    root = Tk()
    root.withdraw()  # Hide the root window
    filepath = filedialog.askopenfilename(title="Open image file")
    return filepath

def get_output_directory():
    # Open dialog to select output directory
    root = Tk()
    root.withdraw()
    directory = filedialog.askdirectory(title="Select output directory for MIB masks")
    return directory

def load_image_stack(image_path):
    # Load the 32-bit TIFF image stack using PIL
    image = Image.open(image_path)
    image_stack = []

    try:
        while True:
            # Load each frame in the TIFF
            frame = np.array(image)
            image_stack.append(frame)
            image.seek(image.tell() + 1)  # Move to next frame
    except EOFError:
        # End of file reached
        pass

    return image_stack

def apply_threshold(image, value):
    # Create a binary mask where the pixel value is equal to the given value
    mask = cv2.inRange(image, np.array(value), np.array(value))
    return mask

def save_image_stack(image_stack, filename):
    # Save the image stack as a multi-page TIFF
    pil_images = [Image.fromarray(img) for img in image_stack]
    pil_images[0].save(filename, save_all=True, append_images=pil_images[1:], compression="tiff_deflate")

def process_image(image_path, output_dir, prefix):
    # Load the image stack (32-bit TIFF image)
    image_stack = load_image_stack(image_path)

    if len(image_stack) == 0:
        print("Failed to load the image stack.")
        return

    # Get unique pixel values in the image stack
    unique_values = np.unique(image_stack)

    # Process each unique pixel value
    for value in unique_values:
        # Create an empty list to store masks for each slice
        mask_stack = []

        # Apply the threshold to each slice in the stack
        for image_slice in image_stack:
            mask = apply_threshold(image_slice, value)
            mask_stack.append(mask)

        # Save the stack of masks as a multi-page TIFF
        index = f"{value:02d}"  # Add leading zero for single-digit numbers
        output_path = os.path.join(output_dir, f"{prefix}_{index}.tif")
        save_image_stack(mask_stack, output_path)

        print(f"Saved: {output_path}")

def main():
    # Step 1: Open the image file
    image_path = open_file_dialog()

    # Step 2: Ask for the filename prefix
    prefix = input("Enter the desired filename prefix (default is 'default'): ") or "default"

    # Step 3: Select output directory
    output_dir = get_output_directory()

    # Step 4: Process the image and save masks
    process_image(image_path, output_dir, prefix)

if __name__ == "__main__":
    main()

2024-09-22 13:05:35.619 Python[4636:43774] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Saved: /Users/siyangc4/Downloads/default_00.tif
Saved: /Users/siyangc4/Downloads/default_01.tif
Saved: /Users/siyangc4/Downloads/default_02.tif
Saved: /Users/siyangc4/Downloads/default_03.tif
